# Collaborative Filtering

In [1]:
import numpy as np
import pandas as pd

In [2]:
books=pd.read_csv('data/BX-Books.csv',on_bad_lines='skip',encoding='latin-1')

C:\Users\shriy\AppData\Local\Temp\ipykernel_11356\779733239.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books=pd.read_csv('data/BX-Books.csv',on_bad_lines='skip',encoding='latin-1')


In [3]:
books.head()

,isbn,book_title,book_author,year_of_publication,publisher
0,195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,2005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,60973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company


In [4]:
books.rename(columns={'book_title':'title','book_author':'author','year_of_publication':'year'},inplace=True)

In [5]:
users=pd.read_csv('data/BX-Users.csv',on_bad_lines='skip',encoding='latin-1')

C:\Users\shriy\AppData\Local\Temp\ipykernel_11356\4223573184.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  users=pd.read_csv('data/BX-Users.csv',on_bad_lines='skip',encoding='latin-1')


In [6]:

users.rename(columns={'Location':'location','Age':'age'},inplace=True)

In [7]:
ratings=pd.read_csv('data/BX-Book-Ratings.csv',on_bad_lines='skip',encoding='latin-1')

In [8]:
ratings.head(2)

,user_id,isbn,rating
0,276725,034545104X,0
1,276726,155061224,5


In [9]:
x=ratings['user_id'].value_counts()>200

In [10]:
x[x].shape #x_filtered=x[x==True], x_filtered.shape
y=x[x].index

In [11]:
y

Index([ 11676, 198711, 153662,  98391,  35859, 212898, 278418,  76352, 110973,
       235105,
       ...
        33145, 116122,  88793, 155916,   9856, 188951,  44296,  59727,  73681,
        28634],
      dtype='int64', name='user_id', length=815)

In [12]:
ratings=ratings[ratings['user_id'].isin(y)]

In [13]:
ratings.shape

(482728, 3)

In [14]:
ratings_with_books=ratings.merge(books,on='isbn')

In [15]:
ratings_with_books.shape

(446881, 7)

In [16]:
number_rating=ratings_with_books.groupby('title')['rating'].count().reset_index()

In [17]:
number_rating.rename(columns={'rating':'number of rating'},inplace=True)

In [18]:
number_rating

,title,number of rating
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,Beyond IBM: Leadership Marketing and Finance ...,1
4,Clifford Visita El Hospital (Clifford El Gran...,1
...,...,...
153247,Ã?Â?ber das Fernsehen.,2
153248,Ã?Â?ber die Pflicht zum Ungehorsam gegen den S...,3
153249,Ã?Â?lpiraten.,1
153250,Ã?Â?rger mit Produkt X. Roman.,1


In [19]:
final_rating=ratings_with_books.merge(number_rating,on='title')

In [20]:
final_rating.shape

(446881, 8)

In [21]:
final_rating=final_rating[final_rating['number of rating']>=50] 
#dataset now has users who hv rated >200 and those books which have been rated by >50 users

In [22]:
final_rating.drop_duplicates(['user_id','title'],inplace=True) #dropping entries where same user has rated same book more than once

In [23]:
final_rating.shape

(49070, 8)

In [24]:
book_pivot=final_rating.pivot_table(columns='user_id',index='title',values='rating')

In [25]:
book_pivot.fillna(0,inplace=True)

In [26]:
from scipy.sparse import csr_matrix
book_sparse=csr_matrix(book_pivot)

In [27]:
from sklearn.neighbors import NearestNeighbors
model=NearestNeighbors(algorithm='brute')

In [28]:
model.fit(book_sparse)

NearestNeighbors(algorithm='brute')

In [29]:
distances,suggestions=model.kneighbors(book_pivot.iloc[54,:].values.reshape(1,-1),n_neighbors=6)

In [30]:
suggestions

array([[ 54, 616, 401, 134, 274, 529]])

In [31]:
for i in suggestions[0][1:]:
    print(book_pivot.index[i])

Winter Moon
Strangers
Dragon Tears
Master of the Game
The Regulators


In [32]:
book_pivot.index[54]

'Bag of Bones'

In [33]:
def book_recommend(book_name):
    book_id = np.where(book_pivot.index==book_name)[0][0]
    distances,suggestions=model.kneighbors(book_pivot.iloc[book_id,:].values.reshape(1,-1),n_neighbors=6)
    for i in range(1,len(suggestions[0])):
            print(book_pivot.index[suggestions[0][i]])



In [ ]:
book_recommend('Bag of Bones')

Winter Moon
Strangers
Dragon Tears
Master of the Game
The Regulators
